# Si twist

In [1]:
from interfacemaster.cellcalc import get_primitive_hkl, rot
from interfacemaster.interface_generator import core, convert_vector_index
from numpy import array, dot, round, var, average, pi
from numpy.linalg import inv
from gb_code.csl_generator import print_list

## Define an interface core class

Input two cif files.For GBs, input the same file; for interfaces, input two different files.


The first structure is non-changed while the second one can be deformed to form an approximate CSL

Please input files of CONVENTIONAL cells if you are not familiar with the indices in the primitive cell.

In [2]:
my_interface = core('cif_files/Si_mp-149_conventional_standard.cif','cif_files/Si_mp-149_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


## Specify the rotation axis and searching parameters to find an approximate CSL

Since this is for a cubic lattice, we have determined rotation axis/angle to give a specific CSL

In [3]:
print_list([1,1,1],100)

Sigma:     1  Theta:   0.00 
Sigma:     3  Theta:  60.00 
Sigma:     7  Theta:  38.21 
Sigma:    13  Theta:  27.80 
Sigma:    19  Theta:  46.83 
Sigma:    21  Theta:  21.79 
Sigma:    31  Theta:  17.90 
Sigma:    37  Theta:  50.57 
Sigma:    39  Theta:  32.20 
Sigma:    43  Theta:  15.18 
Sigma:    49  Theta:  43.57 
Sigma:    57  Theta:  13.17 
Sigma:    61  Theta:  52.66 
Sigma:    67  Theta:  24.43 
Sigma:    73  Theta:  11.64 
Sigma:    79  Theta:  33.99 
Sigma:    91  Theta:  10.42 
Sigma:    93  Theta:  42.10 
Sigma:    97  Theta:  30.59 


In [4]:
#rotation matrix
R = rot([1,1,1],60.00/180*pi)

In [5]:
#take this as an 'exact' CSL
my_interface.search_fixed(R, exact = True, tol = 1e-3)

Congrates, we found an appx CSL!

U1 = 
[[ 0  1 -1]
 [ 1 -1 -1]
 [-1  0 -1]]; sigma_1 = 3

U2 = 
[[-1  1 -1]
 [ 1  0 -1]
 [ 0 -1 -1]]; sigma_2 = 3

D = 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]



'U1', 'U2' are two sets of indices of the CSL in the two crystals' frames,
'D' is the deformation applied to the crystal 2 to form an approximate CSL

As you can see, to form a twinning structure we need to deform the structure provided!

## Convert the miller indices to be expressed in the primitive cell

Now we need to specify the orientation of our interface, which is by giving a miller indices in expressed in the primitive cell.

In [6]:
"""
The interface plane is the [1,0,0] plane in the conventional cell and now we \
transform it into primitive cell
"""
hkl = get_primitive_hkl(array([1,1,1]), my_interface.conv_lattice_1, my_interface.lattice_1)
hkl

array([-1, -1, -1])

## Compute the indices of the two slabs to make bicrystal

Now let's compute the indices of the two slabs forming the interface
If you want a near orthogonal bicrystal, please set 'orthogonal to yes, 
and adjust the lim & tol.

In [7]:
my_interface.compute_bicrystal(hkl, normal_ortho = True, plane_ortho = True, lim = 50, tol_ortho = 1e-3, tol_integer = 1e-3)

cell 1:
[[ 1  1  1]
 [ 1 -1  1]
 [ 1  0 -2]]
cell 2:
[[ 1  1 -1]
 [ 1  0  2]
 [ 1 -1 -1]]


Here we obtained the two sets of indices to build a bicrystal

## Make a bicrystal!

You can adjust the bicrystal by
1. do expansion by set 'xyz' = [dimX, dimY, dimZ]
2. do RBT by 'dzdy' = a translation vector 
(please confine dzdy in the interface plane, 
for RBT in the normal, please use 'dp1' and 'dp2', to make vacuum in the interface use 'dx');
3. introduce vacuum in the interface by 'dx' = the distance spacing the two crystals
4. adjust the termination by 'dp1', 'dp2' = length of shift of the position of 
the termination from the interface into the bulk
5. introduce vacuum to make a surface in the end of the bicrystal by
'vx' = length of the vaccumn

default: xyz = [1,1,1] and others = 0

In [10]:
my_interface.get_bicrystal(xyz_1 = [1,1,1], xyz_2 = [1,1,1], filetype='VASP', dp1 = 0, dp2 = 0)

Good! You have made a 'POSCAR' file of this bicrystal, we also generated two files named 'cell_1.cif' and 'cell_2.cif' providing the structure of the two slabs.

In [11]:
import shutil
try:
    shutil.rmtree('CNID_inputs')
except:
    print('No existing files')
#To sample the CNID, try
my_interface.sample_CNID(grid = [4,4], xyz_1 = [1,1,1], xyz_2 = [1,1,1])

CNID
[[-0.  1.]
 [ 1. -0.]
 [-1. -1.]]
making 16 files...
completed


Please check the generated 'POSCAR.x.y' files. 
The CNID here is expressed in the primitive cell's frame.
You can refer to the cartesian coordinates or in the conventional cell's frame

In [12]:
CNID_cartesian = round(my_interface.CNID,8)
CNID_cartesian

array([[-2.734364, -2.734364],
       [ 2.734364,  0.      ],
       [ 0.      ,  2.734364]])

In [13]:
CNID_conv = round(dot(inv(my_interface.conv_lattice_1)\
                              , my_interface.CNID),8)
CNID_conv

array([[-0.5, -0.5],
       [ 0.5,  0. ],
       [-0. ,  0.5]])